In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import json
import psycopg2

In [10]:
def read_db_water():
    with open('db_config.json') as file:
        db_config = json.load(file)

    engine = create_engine(f'postgresql+psycopg2://{db_config["user"]}:{db_config["password"]}@{db_config["host"]}:5432/{db_config["dbname"]}')

    water = pd.read_sql('SELECT * FROM water_table', engine)

    print(water.head())

read_db_water()


          Año NombreDepartamento  Div_dpto NombreMunicipio  Divi_muni  \
0  2010-01-01            Bolívar        13        El Guamo      13248   
1  2010-01-01            Bolívar        13        El Guamo      13248   
2  2010-01-01            Bolívar        13        El Guamo      13248   
3  2010-01-01            Bolívar        13        El Guamo      13248   
4  2010-01-01            Bolívar        13        El Guamo      13248   

   IrcaMinimo  IrcaMaximo  IrcaPromedio NombreParametroAnalisis2  \
0         0.0       100.0         37.32        Alcanilidad Total   
1         0.0       100.0         37.32                 Aluminio   
2         0.0       100.0         37.32                 Arsénico   
3         0.0       100.0         37.32                   Cadmio   
4         0.0       100.0         37.32                   Calcio   

   MuestrasEvaluadas  MuestrasTratadas  MuestrasSinTratar  \
0                 67                67                  0   
1                 67          

In [12]:
def read_db_water():
    with open('db_config.json') as file:
        db_config = json.load(file)

    engine = create_engine(f'postgresql+psycopg2://{db_config["user"]}:{db_config["password"]}@{db_config["host"]}:5432/{db_config["dbname"]}')

    water = pd.read_sql('SELECT * FROM water_table', engine)
    
    return water

In [14]:
from transform_dag import apply_transformations

def process_water_data():
    # Leer datos de la base de datos
    water_df = read_db_water()
    
    # Aplicar transformaciones
    transformed_water_df = apply_transformations(water_df)
    
    # Aquí podrías agregar más lógica, como guardar los resultados, enviarlos a otro sistema, etc.
    print(transformed_water_df.head())

# Llamar a la función principal para procesar los datos
process_water_data()


2024-05-03 10:28:09,455 - INFO - Starting transformations on water data.
2024-05-03 10:28:09,772 - INFO - Standardized place names.
2024-05-03 10:28:09,805 - INFO - Scaled numerical columns.
2024-05-03 10:28:09,809 - INFO - Column names standardized
2024-05-03 10:28:09,810 - INFO - Standardized column names.


KeyError: 'NombreParametroAnalisis2'